# Stock Market Prediction - Starter Kernel
### Created by Magichanics


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from itertools import chain
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error # wouldn't recommend since we're not being evaluated on MSE
from pandas.tseries.holiday import USFederalHolidayCalendar
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from datetime import datetime
import gc

### Importing Dataframes

In [4]:
from kaggle.competitions import twosigmanews
# You can only call make_env() once, so don't lose it!
env = twosigmanews.make_env()
print('Done!')

Loading the data... This could take a minute.
Done!
Done!


In [9]:
(market_train_df, news_train_df) = env.get_training_data()

In [10]:
# decide the length of the dataset
# note if sampling, len(news_train_df) > len(market_train_df)
market_train_df = market_train_df.tail(100_000)
news_train_df = news_train_df.tail(300_000)

In [11]:
market_train_df.head()

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe
3972956,2016-10-13 22:00:00+00:00,LGIH.O,LGI Homes Inc,365316.0,32.18,32.25,-0.018603,-0.015568,-0.013309,-0.006846,-0.127440,-0.129555,-0.101458,-0.065513,-0.083783,0.0
3972957,2016-10-13 22:00:00+00:00,LGND.O,Ligand Pharmaceuticals Inc,259858.0,95.94,95.23,0.002822,-0.033688,0.004520,-0.034014,-0.059504,-0.068017,-0.092553,-0.145017,-0.040979,0.0
3972958,2016-10-13 22:00:00+00:00,LH.N,Laboratory Corporation of America Holdings,475471.0,136.37,135.79,-0.001099,-0.005347,0.002006,-0.001840,0.006718,-0.008108,0.016626,0.011074,-0.100974,1.0
3972959,2016-10-13 22:00:00+00:00,LHO.N,LaSalle Hotel Properties,939853.0,24.55,24.54,-0.001221,0.007803,0.002493,0.009747,0.030214,0.030659,0.041111,0.057821,-0.041216,1.0
3972960,2016-10-13 22:00:00+00:00,LII.N,Lennox International Inc,263800.0,159.07,158.46,-0.000377,-0.006645,0.002357,-0.003149,0.019157,0.005712,0.029299,0.028708,-0.090346,0.0


Volume is one huge important factor when it comes to predicting stocks.

In [12]:
market_train_df.corr()['returnsOpenNextMktres10']

volume                      0.031921
close                      -0.009862
open                       -0.009662
returnsClosePrevRaw1       -0.001299
returnsOpenPrevRaw1         0.004522
returnsClosePrevMktres1     0.005542
returnsOpenPrevMktres1      0.024243
returnsClosePrevRaw10       0.003571
returnsOpenPrevRaw10       -0.003730
returnsClosePrevMktres10    0.098114
returnsOpenPrevMktres10     0.096051
returnsOpenNextMktres10     1.000000
universe                    0.060272
Name: returnsOpenNextMktres10, dtype: float64

In [13]:
news_train_df.head()

,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetCodes,assetName,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
9028750,2016-08-18 19:30:30+00:00,2016-08-18 19:30:30+00:00,2016-08-18 19:30:30+00:00,a97c51d615a5b59d,"DGAP-PVR: Diebold, Inc.: Release according to ...",3,1,EQS,"{'NEWR', 'EQS', 'COMP', 'TMT', 'INDS', 'TECH',...","{'CNRE', 'CNR', 'RCE'}",5418,1,,False,38,728,{'DBD.N'},Diebold Nixdorf Inc,1,1.000000,0,0.165623,0.612871,0.221506,254,1,1,1,1,2,1,1,6,13,14
9028751,2016-08-18 19:32:40+00:00,2016-08-18 19:32:40+00:00,2016-08-18 19:32:40+00:00,c615f885535cc1a0,"T-Mobile, Sprint roll out revamped unlimited d...",3,1,RTRS,"{'TMT', 'US', 'TCOM', 'RTRS', 'CMPNY', 'AMERS'...","{'PCO', 'PCU', 'DNP', 'PSC', 'U', 'RNP', 'NAW'...",2754,2,,False,21,493,"{'S.N', 'S.MX'}",Sprint Corp,1,1.000000,1,0.174365,0.394979,0.430657,383,0,0,0,0,0,3,4,9,10,11
9028752,2016-08-18 19:32:40+00:00,2016-08-18 19:32:40+00:00,2016-08-18 19:32:40+00:00,c615f885535cc1a0,"T-Mobile, Sprint roll out revamped unlimited d...",3,1,RTRS,"{'TMT', 'US', 'TCOM', 'RTRS', 'CMPNY', 'AMERS'...","{'PCO', 'PCU', 'DNP', 'PSC', 'U', 'RNP', 'NAW'...",2754,2,,False,21,493,"{'TMUS.O', 'TMUS.OQ'}",T-Mobile US Inc,1,1.000000,1,0.272124,0.351084,0.376792,439,0,0,0,0,0,4,4,16,17,17
9028753,2016-08-18 19:35:33+00:00,2016-08-18 19:35:33+00:00,2016-08-18 19:35:33+00:00,f5a82e5d2267afa7,CORRECTED-BREAKINGVIEWS-Feds mete out justice ...,3,2,RTRS,"{'BRV', 'FINS', 'RTRS', 'CLJ', 'PRIV', 'BACT',...","{'T', 'PSC', 'U', 'D', 'M', 'NAW', 'E', 'NAT'}",4613,2,CORRECTED,False,36,921,{'GEO.N'},Geo Group Inc,9,0.377964,-1,0.706255,0.205329,0.088416,94,1,1,1,1,1,12,12,12,12,12
9028754,2016-08-18 19:35:33+00:00,2016-08-18 19:35:33+00:00,2016-08-18 19:35:33+00:00,f5a82e5d2267afa7,CORRECTED-BREAKINGVIEWS-Feds mete out justice ...,3,2,RTRS,"{'BRV', 'FINS', 'RTRS', 'CLJ', 'PRIV', 'BACT',...","{'T', 'PSC', 'U', 'D', 'M', 'NAW', 'E', 'NAT'}",4613,2,CORRECTED,False,36,921,{'CXW.N'},CoreCivic Inc,8,0.755929,-1,0.465562,0.375727,0.158712,110,1,1,1,1,1,11,11,16,16,20


### Joining Market & News Data

In [7]:
#tempcode
# decide the length of the dataset
# note if sampling, len(news_train_df) > len(market_train_df)
(market_train_df, news_train_df) = env.get_training_data()
market_train_df = market_train_df.tail(100)
news_train_df = news_train_df.tail(300)

In [8]:
def join_market_news(market_df, news_df):
    
    # ERROR: REDUCES LENGTH OF DATAFRAME HERE
    print('market_df :' + str(market_df.shape))
    
    # Fix asset codes (str -> list)
    news_df['assetCodes'] = news_df['assetCodes'].str.findall(f"'([\w\./]+)'")

    # Expand assetCodes
    assetCodes_expanded = list(chain(*news_df['assetCodes']))
    assetCodes_index = news_df.index.repeat( news_df['assetCodes'].apply(len) )

    assert len(assetCodes_index) == len(assetCodes_expanded)
    df_assetCodes = pd.DataFrame({'level_0': assetCodes_index, 'assetCode': assetCodes_expanded})
    
    # create dataframe based on groupby
    news_df_expanded = pd.merge(df_assetCodes, news_df, left_on='level_0', right_index=True, suffixes=(['','_old']))
    
    # create dict of values based on columns
    str_cols = ['time', 'assetCode', 'headline']
    news_data_dict = {}
    news_groupby = news_df_expanded.groupby(['time', 'assetCode'])
    for f in str_cols:
        news_data_dict[f] = news_groupby[f].apply(lambda x: x)
        print('finished ' + f)
    
    news_df_grouped = pd.DataFrame(news_data_dict)
    return news_df_grouped
    
    
#     news_df_group = pd.DataFrame({'headlines' : news_groupby['headline'].apply(lambda x: x),
#                                   'wordCount' : news_groupby['wordCount'].apply(lambda x: x)})
    
#     return news_df_grouped
    
    
    X = market_train_df.join(news_df_grouped, on=['time', 'assetCode'])
    
    # cleanup time
    del market_df
    del news_df
    gc.collect()
    
    print('X shape :' + str(X.shape))
    
    return X

join_market_news(market_train_df, news_train_df)

# headlines                                          wordCount
# 0     ((2016-12-29 11:30:02+00:00, GPT), [GRAMERCY P...           ((2016-12-29 11:30:02+00:00, GPT), [15])
# 1     ((2016-12-29 11:30:02+00:00, GPT.N), [GRAMERCY...         ((2016-12-29 11:30:02+00:00, GPT.N), [15])
# 2     ((2016-12-29 11:30:07+00:00, GPT), [GRAMERCY P...           ((2016-12-29 11:30:07+00:00, GPT), [10])
# 3     ((2016-12-29 11:30:07+00:00, GPT.N), [GRAMERCY...         ((2016-12-29 11:30:07+00:00, GPT.N), [10])
# 4     ((2016-12-29 11:30:49+00:00, HYG.TO), [HYDROGE...         ((2016-12-29 11:30:49+00:00, HYG.TO), [9])
# 5     ((2016-12-29 11:30:49+00:00, HYGS.O), [HYDROGE...         ((2016-12-29 11:30:49+00:00, HYGS.O), [9])
# 6     ((2016-12-29 11:30:49+00:00, HYGS.OQ), [HYDROG...        ((2016-12-29 11:30:49+00:00, HYGS.OQ), [9])

market_df :(100, 16)
finished time
finished assetCode
finished headline


,time,assetCode,headline
0,2016-12-30 16:19:08+00:00,MNR.N,BRIEF-Monmouth Real Estate buys new building i...
1,2016-12-30 16:19:42+00:00,JNJ.DE,BUZZ-Big 2016 healthcare stock losers tally la...
2,2016-12-30 16:19:42+00:00,JNJ.N,BUZZ-Big 2016 healthcare stock losers tally la...
3,2016-12-30 16:19:42+00:00,AGN.N,BUZZ-Big 2016 healthcare stock losers tally la...
4,2016-12-30 16:19:42+00:00,ENDP.OQ,BUZZ-Big 2016 healthcare stock losers tally la...
5,2016-12-30 16:19:42+00:00,ENDP.O,BUZZ-Big 2016 healthcare stock losers tally la...
6,2016-12-30 16:19:42+00:00,MYL.OQ,BUZZ-Big 2016 healthcare stock losers tally la...
7,2016-12-30 16:19:42+00:00,MYL.O,BUZZ-Big 2016 healthcare stock losers tally la...
8,2016-12-30 16:19:42+00:00,ABT.N,BUZZ-Big 2016 healthcare stock losers tally la...
9,2016-12-30 16:24:45+00:00,ETRM.O,ENTEROMEDICS INC SHARES DOWN 17.3 PCT AT $2.02...


In [ ]:
X_train.head()

### Aggregations

In [ ]:
def aggregations(df):
    
    # get columns
    lst_of_lst_of_cols = [[f for f in df.columns if 'return' in f and f != 'returnsOpenNextMktres10'],
                              [f for f in df.columns if 'novelty' in f],
                              [f for f in df.columns if 'volume' in f],
                              [f for f in df.columns if 'sentiment' in f]]
    
    agg_suffixes = ['aggReturn ', 'aggNovelty ', 'aggVolume ', 'aggSentiment ']
    
    for i_cols in range(len(lst_of_lst_of_cols)):
        
        # setup map of aggregations
        agg_dict = {}
        for col in lst_of_lst_of_cols[i_cols]:
            agg_dict[col] = ['mean', 'var', 'sum', 'std', 'max', 'min']
            
        # preform aggregations
        df_agg = df.groupby('sourceId').agg(agg_dict)
        df_agg.columns = pd.Index(['agg_' + e[0] + "_" + e[1].lower() for e in df_agg.columns.tolist()])
        
        # clean up dataframe and merge
        df = df.join(df_agg, how = 'left', on = 'sourceId', lsuffix = agg_suffixes[i_cols])
        del df_agg
        gc.collect()
        
        print('finished ' + agg_suffixes[i_cols])
        
    print('New dataframe shape: ' + str(df.shape))
    return df

X_train = aggregations(X_train)

### Text Processing with MultinomialNB (Headlines)

In [ ]:
def get_headline(headlines_df):
    
    # get headlines as list
    headlines_lst = []
    for row in range(0,len(headlines_df.index)):
        headlines_lst.append(headlines_df.iloc[row])

    # split headlines to separate words
    basicvectorizer = CountVectorizer()
    headlines_vectorized = basicvectorizer.fit_transform(headlines_lst)
    
    print(headlines_vectorized.shape)
    return headlines_vectorized, basicvectorizer

def headline_mapping(target, headlines_vectored, headline_vectorizer):
    
    # round target values if using logistic regression
    target = round(target,0)
    
    # get model (testing with model that isn't )
    from sklearn.naive_bayes import MultinomialNB
    headline_model = MultinomialNB()
    headline_model = headline_model.fit(headlines_vectored, target)
    
    # get coefficients
    basicwords = headline_vectorizer.get_feature_names()
    basiccoeffs = headline_model.coef_.tolist()[0]
    coeff_df = pd.DataFrame({'Word' : basicwords, 
                            'Coefficient' : basiccoeffs})
    
    # convert dataframe to dictionary of coefficients
    coefficient_dict = dict(zip(coeff_df.Word, coeff_df.Coefficient))

    return coefficient_dict, coeff_df['Coefficient'].mean()

def get_coeff_col(headlines_df, coeff_dict, coeff_default):
    
    def get_coeff(word_lst):
        
        # iter through every word
        coeff_sum = 0
        for word in word_lst:
            if word in coeff_dict:
                coeff_sum += coeff_dict[word]
            else:
                coeff_sum += coeff_default
        
        # get average coefficient
        return coeff_sum / len(word_lst)
        
    basicvectorizer = CountVectorizer()
    
    # loop through every item
    headlines_coeff_lst = []
    for row in range(0,len(headlines_df.index)):
        headlines_coeff_lst.append(get_coeff(str(headlines_df.iloc[row]).split(' ')))
    
    return pd.Series(headlines_coeff_lst)

coefficient_dict, coefficient_default = headline_mapping(X_train['returnsOpenNextMktres10'],
                                            *get_headline(X_train['headline']))

X_train['headline_coeff_mean'] = get_coeff_col(X_train['headline'], coefficient_dict, coefficient_default)

In [ ]:
X_train['headline_coeff_mean'].head()

### Merge Dataframes

### Get Time Features

In [ ]:
# ripped from my previous kernel, NYC Taxi Fare

# first get dates
def split_time(df):
    
    # convert to string (will find a more efficient way to do this without converting to string)
    df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # split date_time into categories
    df['time_day'] = df['time'].str.slice(8,10)
    df['time_month'] = df['time'].str.slice(5,7)
    df['time_year'] = df['time'].str.slice(0,4)
    df['time_hour'] = df['time'].str.slice(11,13)
    
    # source: https://www.kaggle.com/nicapotato/taxi-rides-time-analysis-and-oof-lgbm
    df['temp_time'] = df['time'].str.replace(" UTC", "")
    df['temp_time'] = pd.to_datetime(df['temp_time'], format='%Y-%m-%d %H:%M:%S')
    
    df['time_day_of_year'] = df.temp_time.dt.dayofyear
    df['time_week_of_year'] = df.temp_time.dt.weekofyear
    df["time_weekday"] = df.temp_time.dt.weekday
    df["time_quarter"] = df.temp_time.dt.quarter
    
    del df['temp_time']
    gc.collect()
    
    # convert to non-object columns
    time_feats = ['time_day', 'time_month', 'time_year', 'time_hour']
    df[time_feats] = df[time_feats].apply(pd.to_numeric)
    
    # determine whether the day is set on a holiday
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start='2007-01-01', end='2018-09-27').to_pydatetime()
    df['on_holiday'] = df['time'].str.slice(0,10).apply(lambda x: 1 if x in holidays else 0)
    
    # note to self: encode time later on
    
    return df

X_train = split_time(X_train)

In [ ]:
def get_misc_features(X_df):
    
    # Adding daily difference
    new_col = X_df["close"] - X_df["open"]
    X_df.insert(loc=6, column="daily_diff", value=new_col)
    X_df['close_to_open'] =  np.abs(X_df['close'] / X_df['open'])

### Label Encoding

In [ ]:
def group_delete(df, del_features):
    for f in del_features:
        del df[f]

def encoding(df, categorical_feats):
    df_encoded = pd.get_dummies(df[categorical_feats])
    df.join(df_encoded, how = 'right')
    group_delete(df, categorical_feats)
    print('new shape: ' + str(df.shape))
    return df

group_delete(X_train, ['time', 'sourceId', 'headline', 'assetCodes'])
X_train = encoding(X_train, [f for f in X_train.columns if X_train[f].dtype == 'object'])

### Cleaning Data

In [ ]:
# will use a more efficient way later on
fcol = [c for c in X_train.columns if c not in ['sourceTimestamp', 'firstCreated', 'returnsOpenNextMktres10', 
                                                'assetName_x', 'universe', 'provider', 'subjects',
                                               'audiences', 'marketCommentary', 'assetName_y', 'sourceTimestamp'
                                               'firstCreated']] #<---- added


### Using LGBM for Modelling

In [ ]:
# prepare x dataframes for modelling/prediction
def convert_to_X(market_obs_df, news_obs_df):
    
    # this repeats everything that was done previously
    X_test = join_market_news(market_obs_df, news_obs_df)
    X_test = aggregations(X_test)
    X_test['headline_coeff_mean'] = get_coeff_col(X_test['headline'], coefficient_dict, coefficient_default)
    X_test = split_time(X_test)
    group_delete(X_test, ['time', 'sourceId', 'headline', 'assetCodes'])
    X_test = encoding(X_test, ['assetCode', 'headlineTag'])
    X_test = X_test[[f for f in X_test.columns if 'int' in str(X_test[f].dtype) or 'float' in str(X_test[f].dtype)]]
    
    return X_test

In [ ]:
y_train = X_train['returnsOpenNextMktres10']
del X_train['returnsOpenNextMktres10']

In [ ]:
import lightgbm as lgb
import time

# set model and parameters
params = {'learning_rate': 0.02, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'seed': 2018}

In [ ]:
#split data (for cross validation)
x1, x2, y1, y2 = train_test_split(X_train[fcol], 
                                  y_train, 
                                  test_size=0.25, 
                                  random_state=99)

In [ ]:
# train
t = time.time()
print('Fitting Up')

# cross validation
lgb_model = lgb.train(params, 
                        lgb.Dataset(x1, label=y1), 
                        5000, 
                        lgb.Dataset(x2, label=y2), 
                        verbose_eval=100, 
                        early_stopping_rounds=200)

# lgb_model = lgb.train(params, 
#                         lgb.Dataset(X_train[fcol], label=y_train),
#                         verbose_eval=100)

print(f'Done, time = {time.time() - t}')

In [ ]:
def make_predictions(market_obs_df, news_obs_df):
    
    print('market_obs_df shape: ' + str(market_obs_df.shape))
    print('news_obs_df shape: ' + str(news_obs_df.shape))
    
    # predict using given model
    X_test = convert_to_X(market_obs_df, news_obs_df)
    print('Created X_test with features: ' + str(X_test[fcol].columns))
    
    # there is an error:
    # ValueError: Length of values does not match length of index
    prediction_values = np.clip(lgb_model.predict(X_test[fcol]), -1, 1)
    
    print('finished predictions')

    return prediction_values

### Making Predictions

Now the difference between the training and test data would be these two columns,  ``['returnsOpenNextMktres10', 'universe']``. We will be trying to predict ``returnsOpenNextMktres10`` and using that as the ``confidenceValue``.

In [ ]:
for (market_obs_df, news_obs_df, predictions_template_df) in env.get_prediction_days(): # Looping over days from start of 2017 to 2019-07-15
    
    print('predictions_template_df shape: ' + str(predictions_template_df.shape))
    # make predictions
    predictions_template_df['confidenceValue'] = make_predictions(market_obs_df, news_obs_df)
    
    # save predictions
    env.predict(predictions_template_df)


### Export Submission

In [ ]:
env.write_submission_file() # Writes your submission file
print('finished!')

### Sources:
* [Getting Started - DJ Sterling](https://www.kaggle.com/dster/two-sigma-news-official-getting-started-kernel)
* [Bare bones script - William Cukierski](https://www.kaggle.com/wcukierski/bare-bones-script-loop-with-comments)
* [Extra data - aaron7sun](https://www.kaggle.com/aaron7sun/stocknews)
* [Text Preprocessing - Andrew Gelé](https://www.kaggle.com/ndrewgele/omg-nlp-with-the-djia-and-reddit)
* [fake news - SamLloyd](https://www.kaggle.com/sjdlloyd/it-s-fake-news-this-is-top-of-the-leaderboard)
* [a simple model - Bruno G. do Amaral](https://www.kaggle.com/bguberfain/a-simple-model-using-the-market-data)
* [LGBM Model - the1owl](https://www.kaggle.com/the1owl/my-two-sigma-cents-only)